# Using `breads` for wavelength and resolution calibration

## Installation

First, ensure that you have a current version of `breads` on your system. The easiest way to do this after release will be:

In [ ]:
# after release

# !pip install breads

# restart your Jupyter kernel

We are currently in pre-release, so you can run one of the following two options. This assumes that you have cloned our [GitHub repository](https://github.com/jruffio/bread) and are running this iPython notebook from the `breads\demos` subdirectory. These options are also useful if you want to test out code that is unreleased on PyPi but on GitHub. 

The first option will affect your Python/conda environment. The second option does not, but you will have to run it every time you try to use `breads` for a demo or your own code.

In [10]:
# pre release

# %cd .. 
# !python setup.py build
# !python setup.py install
# %cd ./demos

# restart your Jupyter kernel

In [2]:
# pre release

# import sys
# sys.path.append('../')

Finally, we import the necessary modules from `breads`

In [3]:
import breads.calibration as cal
import breads.instruments.OSIRIS as os
import time
import matplotlib.pyplot as plt
import astropy.io.fits as pyfits

# Tutorial

# OH Line Calibration or Sky Calibration

`breads.calibration` uses an image of the sky to return corrected values of wavelengths and resolution.

* To run this tutorial, you will need to download a sample sky present [here](https://drive.google.com/file/d/1yljrw1MvA7as2BZG86pQslYp_GUTw3Xt/view?usp=sharing). Additional sample skies are present as `.fits` files in `public_osiris_data/kap_And/20161106/skies` available [here](https://drive.google.com/drive/folders/1zM8pfqMRQQovq-2wFC6yIguhNWZiP9Nd?usp=sharing). You can also use your own sky images obtained using Keck/OSIRIS.

* Place this file in the same directory (`demos`) as this iPython notebook. Update the filename with path below.

In [4]:
filename = './s161106_a024002_Kbb_020.fits'
obj = os.OSIRIS(filename)

We select a smaller 4 x 5 spatial region of the full 64 X 19 data, as running the calibration on full file takes significant time.

In [5]:
obj.data = obj.data[:, 20:24, 5:10]

In [6]:
start = time.time()
SkyCalibrationObj = cal.sky_calibration(obj, R=None)
print("time taken (s):", time.time() - start)

time taken (s): 51.75033974647522


When running through command line, `breads.calibration` will give you updates on progress in the form `row: {row}, col: {col}`, but the combination of multiprocessing and iPython are supressing that output for specifically this notebook. If you are using Jupyter, you will see this output on the terminal where you ran `jupyter notebook` instead.

`fit_values` are such that $\lambda_{new} = \lambda_{old} (1+a_1) + a_0$ (by default). There is an option to center data (`center_data` optional argument), in which case, $\lambda_{new} = (\lambda_{old} - \bar{\lambda}_{old}) a_1 + a_0$.

Return value is a `SkyCalibration` object that contains the needed information.

In [7]:
print("shape of corrected wavelengths: ", SkyCalibrationObj.corrected_wavelengths.shape)
print("new: ", SkyCalibrationObj.corrected_wavelengths[:, 0, 0])
print("old: ", obj.wavelengths)
print("filename: ", SkyCalibrationObj.calib_filename)
print("unit: ", SkyCalibrationObj.unit)
print("shape of all fit values (see below): ", SkyCalibrationObj.fit_values.shape)


shape of corrected wavelengths:  (1665, 4, 5)
new:  [1.9649637 1.9652137 1.9654636 ... 2.380416  2.3806658 2.3809159]
old:  [1.965   1.96525 1.9655  ... 2.3805  2.38075 2.381  ]
filename:  ./calib_file.fits
unit:  (Unit("Angstrom"), None, None)
shape of all fit values (see below):  (4, 5, 3)


In [8]:
%matplotlib widget 
fig, ax = plt.subplots()
im = ax.imshow(SkyCalibrationObj.fit_values[:, :, 0])
cbar = fig.colorbar(im, extend='max')
plt.title(r"constant offset $a_0$")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
%matplotlib widget 
fig, ax = plt.subplots()
im = ax.imshow(SkyCalibrationObj.fit_values[:, :, 1])
cbar = fig.colorbar(im, extend='max')
plt.title(r"linear offset $a_1$")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
%matplotlib widget 
fig, ax = plt.subplots()
im = ax.imshow(SkyCalibrationObj.fit_values[:, :, 2])
cbar = fig.colorbar(im, extend='max')
plt.title(r"Resolution $R$")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
plt.close('all')

This calibration data is automatically saved to a `.fits` file. The name of the file can be passed as `calib_filename` to `breads.calibration.sky_calibration`. It defaults to `./calib_file.fits`.

In [12]:
with pyfits.open('./sky_calib_file.fits') as hdulist:
    print(len(hdulist))
    for hdu in hdulist:
        print(hdu.header["TYPE"], hdu.data.shape)

4
corrected_wavelengths (1665, 4, 5)
const (4, 5)
RV (4, 5)
R (4, 5)


## Restoration
If you used the first two of the three options at the start of the tutorial to install `breads` on your system and now wish to remove the install, simply run the cell below.

In [ ]:
# !pip uninstall -y breads